# Distillation 

Ce a pour but d'entrainer un MPS multicouche à l'aide d'un réseau complètement connecté

In [40]:
%pip install "git+https://github.com/jemisjoky/TorchMPS.git"
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
%pip install torchmetrics
from torchmetrics.classification import MulticlassAccuracy
from torchmps import MPS

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/jemisjoky/TorchMPS.git to c:\users\piche\appdata\local\temp\pip-req-build-4cu5hy2f
  Resolved https://github.com/jemisjoky/TorchMPS.git to commit 6c0bc1a8e2c15acba8570ca9ffe2b4a0c7135165
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/jemisjoky/TorchMPS.git 'C:\Users\piche\AppData\Local\Temp\pip-req-build-4cu5hy2f'


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [41]:
# Hardware hyperparameters
chosen_device = torch.device('cuda' 
if torch.cuda.is_available() else 'cpu')

# Data hyperparameters
nb_train = 30
nb_test = 50
chosen_bs = 100
chosen_bs = min(nb_train, chosen_bs)
input_size = 28*28
nb_classes = 10

# Teacher hyperparameters
chosen_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n_epochs_fcnn = 20
hidden_size = 70
teacher_loss = nn.CrossEntropyLoss()
#Optimizer parameters
chosen_lr = 0.001
chosen_momentum = 0.9

# Student hyperparameters
# MPS parameters
bond_dim = 20
adaptive_mode = False
periodic_bc = False
feature_map = lambda x : torch.tensor([1, x]).to(chosen_device)
feature_dim = 2
# Training parameters
gauss_epochs = 5 # number of epochs with added gaussian noise
gn_var = 0.3 #added gaussian noise variance
gn_mean = 0 #added gaussian noise mean
n_epochs_lmps = 30
mps_learn_rate = 0.0001
mps_reg = 0.0
mps1_distill_loss = nn.MSELoss()
# We choose this loss 
mps2_distill_loss = nn.KLDivLoss(reduction = "batchmean", log_target = True)

In [42]:
# Import the mnist dataset
train_set = torchvision.datasets.MNIST(root = './datasets', train = True,   
    transform = transforms.ToTensor(),  download = True )

train_subset = torch.utils.data.SubsetRandomSampler(range(nb_train))

train_iterator = torch.utils.data.DataLoader(dataset = train_set, 
    sampler = train_subset, batch_size=chosen_bs)


test_set = torchvision.datasets.MNIST(root = './datasets',
 train = False, transform = transforms.ToTensor(),  download = True)

test_subset = torch.utils.data.SubsetRandomSampler(range(nb_test))

test_iterator = torch.utils.data.DataLoader(dataset = test_set, 
    sampler = test_subset, batch_size = chosen_bs)

In [43]:

# Create the fcnn class
class FCNN(nn.Module):
    def __init__(self):
        super(FCNN, self).__init__()
        self.relu = nn.ReLU()
        self.lin1 = nn.Linear(784, hidden_size)
        self.lin2 = nn.Linear(hidden_size, hidden_size)
        self.lin3 = nn.Linear(hidden_size, hidden_size)
        self.lin4 = nn.Linear(hidden_size, hidden_size)
        self.lin5 = nn.Linear(hidden_size, hidden_size)
        self.lin6 = nn.Linear(hidden_size, 10)

    def middleforward(self, x):
        y = self.lin1(x)
        y = self.relu(y)
        y = self.lin2(y)
        y = self.relu(y)
        y = self.lin3(y)
        y = self.relu(y)
        return y

    def forward(self, x):
        y = self.lin1(x)
        y = self.relu(y)
        y = self.lin2(y)
        y = self.relu(y)
        y = self.lin3(y)
        y = self.relu(y)
        y = self.lin4(y)
        y = self.relu(y)
        y = self.lin5(y)
        y = self.relu(y)
        y = self.lin6(y)
        y = self.relu(y)
        return y

#Instantiate and put the model on the chosen device
fcnn_teacher = FCNN().to(chosen_device)

#Instantiate the optimizer
optimizer = torch.optim.Adam(fcnn_teacher.parameters())

#Training loop
for epoch in range(n_epochs_fcnn):
    for (x_mb, y_mb) in train_iterator:
        # Reshape the train_tuple and put on the chosen device
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)
        y_mb = y_mb.to(chosen_device)
        # Foward propagation
        y_hat_mb = fcnn_teacher(x_mb)
        # We use soft-cross-entropy
        # Softmax is precomputed in the loss already in PyTorch
        # Ground truth is scalar tensor
        loss = teacher_loss(y_hat_mb, y_mb)
        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    
    print(loss.item())

# get accuracy
# Get the validation set classification accuracy
teacher_acc_score = 0
teacher_acc_metric = MulticlassAccuracy(num_classes=nb_classes).to(chosen_device)
for (x_mb, y_mb) in test_iterator:
    x_mb = x_mb.reshape(-1, 784).to(chosen_device)
    y_mb = y_mb.to(chosen_device)
    # add the number of datapoints we classified right
    teacher_acc_score += x_mb.size()[0] * teacher_acc_metric( fcnn_teacher(x_mb), y_mb )
print("The teacher's accuracy score is:")
print(teacher_acc_score / nb_test) #divide by total size



2.2990503311157227
2.2970781326293945
2.2953126430511475
2.2936489582061768
2.2915563583374023
2.289383888244629
2.2869019508361816
2.2840330600738525
2.2807462215423584
2.276853084564209
2.2721962928771973
2.2668979167938232
2.2608144283294678
2.253469228744507
2.2445173263549805
2.2335546016693115
2.22001051902771
2.203119993209839
2.182532548904419
2.1575875282287598
The teacher's accuracy score is:
tensor(0.2543)


In [44]:

# Initialize the MPS modules
student_mps1 = MPS(
    input_dim = input_size,
    feature_dim = feature_dim,
    output_dim = hidden_size,
    bond_dim = bond_dim,
    adaptive_mode = adaptive_mode,
    periodic_bc = periodic_bc,
).to(chosen_device)
student_mps1.register_feature_map(feature_map)

student_mps2 = MPS(
    input_dim = hidden_size, #crucial setting
    feature_dim = feature_dim,
    output_dim = nb_classes,
    bond_dim = bond_dim,
    adaptive_mode = adaptive_mode,
    periodic_bc = periodic_bc,
).to(chosen_device)
student_mps2.register_feature_map(feature_map)

def student(x):
    return student_mps2( student_mps1(x) )


# Instantiate the optimizer and softmax
lmps1_optimizer = torch.optim.Adam(student_mps1.parameters(), 
    lr = mps_learn_rate, weight_decay = mps_reg)

lmps2_optimizer = torch.optim.Adam(student_mps2.parameters(), 
    lr = mps_learn_rate, weight_decay = mps_reg)


# Training loop for the student_mps1
for epoch in range(n_epochs_lmps):
    for (x_mb, _) in train_iterator:

        # Reshape put on the chosen device
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)

        # Added gaussian noise for the last epochs
        if epoch > (n_epochs_lmps - gauss_epochs):
          x_mb = x_mb + torch.randn(size=x_mb.size()).to(chosen_device)

        # Get output of teacher in middlelayer
        # Softmax is precomputed in the loss already
        y_mb = fcnn_teacher.middleforward(x_mb).to(chosen_device)

        # Foward propagation
        y_hat_mb = student_mps1(x_mb)
        # We use mean squared error loss
        loss = mps1_distill_loss(y_hat_mb, y_mb)

        # Backpropagation
        loss.backward()
        lmps1_optimizer.step()
        lmps1_optimizer.zero_grad()

    print(loss.item())

softmax = torch.nn.Softmax(dim=0)

# Training loop for the student_mps2
for epoch in range(n_epochs_lmps):
    for (x_mb, _) in train_iterator:
        # Reshape put on the chosen device
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)

        # Added gaussian noise
        if epoch > (n_epochs_lmps - gauss_epochs):
          x_mb = x_mb + torch.randn(size=x_mb.size()).to(chosen_device)

        # Get log_softmax of logit outputs of teacher
        teacher_output = torch.logit(fcnn_teacher(x_mb))
        teacher_output = nn.functional.log_softmax( teacher_output ) 
        # Get Log_softmax of logit outputs of student
        student_output = torch.logit( student(x_mb) )
        student_output = nn.functional.log_softmax( student_output ) 

        # Chosen loss is Kull divergence
        loss = mps2_distill_loss(student_output, teacher_output)
        

        # Backpropagation
        loss.backward()
        lmps2_optimizer.step()
        lmps2_optimizer.zero_grad()

    print(loss.item())



# Get the validation set classification accuracy
student_acc_score = 0
student_acc_metric = MulticlassAccuracy(num_classes=nb_classes).to(chosen_device)
for (x_mb, y_mb) in test_iterator:
    x_mb = x_mb.reshape(-1, 784).to(chosen_device)
    y_mb = y_mb.to(chosen_device)
    # add the number of datapoints we classified right
    student_acc_score += x_mb.size()[0] * student_acc_metric( student(x_mb), y_mb )
print("Student_acc_score:")
print(student_acc_score / nb_test) #divide by total size


0.7069289088249207
0.48772984743118286
0.15165264904499054
0.30506619811058044
0.11751377582550049
0.18199242651462555
0.16289347410202026
0.12855762243270874
0.10400985926389694
0.0929543673992157
0.09451176226139069
0.10227023810148239
0.10540786385536194
0.10220146179199219
0.09762468934059143
0.09436020255088806
0.0925477147102356
0.09168187528848648
0.09134026616811752
0.09126687049865723
0.09131640195846558
0.09140515327453613
0.09148344397544861
0.09152237325906754
0.0915091410279274
0.09144286811351776
0.10031814128160477
0.11350849270820618
0.1051051914691925
0.1146579459309578


C:\Users\piche\AppData\Local\Temp\ipykernel_12016\3631964471.py:75: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  teacher_output = nn.functional.log_softmax( teacher_output )
C:\Users\piche\AppData\Local\Temp\ipykernel_12016\3631964471.py:78: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  student_output = nn.functional.log_softmax( student_output )


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
Student_acc_score:
tensor(0.1000)
